In [1]:
import sys
import os
from pathlib import Path
from unittest.mock import patch
project_dir = Path(os.path.abspath('')).parent
sys.path.insert(0, project_dir.as_posix())

from tqdm import trange, tqdm
import numpy as np
import torch
from matplotlib import  pyplot as plt
import pandas as pd
from transformers import AutoModel, AutoModelForCausalLM
from vllm import LLM, SamplingParams
from vllm.model_executor import set_random_seed

%load_ext autoreload
%autoreload 2
from cs336_alignment import  train_llm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%matplotlib inline

INFO 11-05 15:09:59 [__init__.py:216] Automatically detected platform cuda.


In [3]:
llm = LLM(
    model=project_dir.joinpath('checkpoints', 'MATH_100_step24').as_posix(),
    gpu_memory_utilization=0.8,
)

INFO 11-05 15:10:44 [utils.py:233] non-default args: {'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': '/workspace/assignment5-alignment/checkpoints/MATH_100_step24'}


INFO 11-05 15:10:44 [model.py:547] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-05 15:10:44 [model.py:1730] Downcasting torch.float32 to torch.float16.
INFO 11-05 15:10:44 [model.py:1510] Using max model len 4096
INFO 11-05 15:10:46 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=10900) INFO 11-05 15:10:47 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=10900) INFO 11-05 15:10:47 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/workspace/assignment5-alignment/checkpoints/MATH_100_step24', speculative_config=None, tokenizer='/workspace/assignment5-alignment/checkpoints/MATH_100_step24', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, s

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=10900) INFO 11-05 15:10:53 [default_loader.py:267] Loading weights took 1.58 seconds
(EngineCore_DP0 pid=10900) INFO 11-05 15:10:54 [gpu_model_runner.py:2653] Model loading took 2.8798 GiB and 1.884412 seconds
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:01 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/e10aa591c0/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:01 [backends.py:559] Dynamo bytecode transform time: 6.43 s
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:04 [backends.py:197] Cache the graph for dynamic shape for later use


(EngineCore_DP0 pid=10900) [rank0]:W1105 15:11:05.579000 10900 site-packages/torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


(EngineCore_DP0 pid=10900) INFO 11-05 15:11:23 [backends.py:218] Compiling a graph for dynamic shape takes 22.23 s
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:30 [monitor.py:34] torch.compile takes 28.66 s in total
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:32 [gpu_worker.py:298] Available KV cache memory: 7.33 GiB
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:32 [kv_cache_utils.py:1087] GPU KV cache size: 274,400 tokens
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:32 [kv_cache_utils.py:1091] Maximum concurrency for 4,096 tokens per request: 66.99x
(EngineCore_DP0 pid=10900) WARNING 11-05 15:11:32 [gpu_model_runner.py:3663] CUDAGraphMode.FULL_AND_PIECEWISE is not supported with FlexAttentionMetadataBuilder backend (support: AttentionCGSupport.NEVER); setting cudagraph_mode=PIECEWISE because attention is compiled piecewise


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:03<00:00, 17.21it/s]


(EngineCore_DP0 pid=10900) INFO 11-05 15:11:37 [gpu_model_runner.py:3480] Graph capturing finished in 5 secs, took 0.40 GiB
(EngineCore_DP0 pid=10900) INFO 11-05 15:11:37 [core.py:210] init engine (profile, create kv cache, warmup model) took 43.56 seconds
INFO 11-05 15:11:38 [llm.py:306] Supported_tasks: ['generate']


In [4]:
sampling_params = SamplingParams(
    temperature=1.0,
    top_p=1.0,
    min_tokens=4,
    max_tokens=1024,
    stop=["</answer>"],
    include_stop_str_in_output=True,
)
path = project_dir.joinpath("data", "MATH", "validation.jsonl")
validation_data = pd.read_json(
    path_or_buf=path,
    lines=True
)
validation_data

,problem,level,type,solution
0,How many vertical asymptotes does the graph of...,Level 3,Algebra,The denominator of the rational function facto...
1,What is the positive difference between $120\%...,Level 1,Algebra,One hundred twenty percent of 30 is $120\cdot3...
2,Find $x$ such that $\lceil x \rceil + x = \dfr...,Level 4,Algebra,"First, we note that $x$ must be positive, sinc..."
3,Evaluate $i^5+i^{-25}+i^{45}$.,Level 5,Algebra,We have $i^5 = i^4\cdot i = 1\cdot (i) = i$. ...
4,"If $2^8=4^x$, what is the value of $x$?",Level 1,Algebra,Rewrite $4$ as $2^2$ to find $4^x=2^{2x}$. Si...
...,...,...,...,...
95,An infinite geometric series has sum 2000. A ...,Level 5,Algebra,Let $a$ be the first term and $r$ the ratio of...
96,Twelve people purchased supplies for a ten-day...,Level 2,Algebra,Since each person of the original group had 10...
97,How many integers belong to the arithmetic seq...,Level 3,Algebra,The common difference is $20 - 13 = 7$. If th...
98,"Of the four points $(2,2)$, $(9,11)$, $(5,7)$,...",Level 3,Algebra,"Consider points $P$, $Q$, and $R$. If the slop..."


In [ ]:
train_llm.evaluate_vllm(llm=llm,
    reward_fn=train_llm.drgrpo_grader.r1_zero_reward_fn,
    prompts=list(validation_data.problem),
    ground_truths=list(validation_data.solution),
    sampling_params=sampling_params,
    save_path=project_dir / "evaluation" / "MATH_100_step24.jsonl",
    overwrite=False
)

Adding requests:   0%|          | 0/100 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

(EngineCore_DP0 pid=10900) CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.
